In [38]:
import numpy as np


class Tensor():
    def __init__(self, value, requires_grad=True):
        self.value = np.array(value, dtype=float)
        self.requires_grad = requires_grad
        self.a, self.b = None, None
        self.dependencies = {self}

        if requires_grad:
            self.grad = np.zeros(self.value.shape)
        
    def backwards(self, queue=[], first=True):
        if first:
            self.grad = np.ones(self.value.shape)

        if self.a is not None and self.a.requires_grad:
            self.backpropagate_a()
            # if not self.a in queue:
            #     queue.append(self.a)
            queue = self.add_to_queue(self.a, queue)

        if self.b is not None and self.b.requires_grad:
            self.backpropagate_b()
            # if not self.b in queue:
            #     queue.append(self.b)
            queue = self.add_to_queue(self.b, queue)
        
        # print(self)
        # print(queue)
        # print(type(self), queue)

        if queue:
            queue[0].backwards(queue=queue[1:], first=False)

    def add_to_queue(self, x, queue):
        for ix, el in enumerate(queue):
            if x == el:
                return queue
            if el in x.dependencies:
                return queue[:ix] + [x] + queue[ix:]
        return queue + [x]

    def backpropagate_a(self):
        pass

    def backpropagate_b(self):
        pass

    def __str__(self):
        return f"({type(self)}, \n value=\n{self.value}, \ngrad=\n{self.grad if self.requires_grad else ''}"

    def __add__(self, b):
        return Add(self, b)

    def __sub__(self, b):
        return Add(self, Neg(b))

    def __mul__(self, b):
        return Mul(self, b)

    def __truediv__(self, b):
        return Mul(self,  b ** Tensor(-1, requires_grad=False)) 

    def __pow__(self, b):
        return Pow(self, b)

    def __matmul__(self, b):
        return Dot(self, b)

    def __gt__(self, b):
        return Tensor(self.value > b.value, requires_grad=False)


class Add(Tensor):
    def __init__(self, a, b):
        super().__init__(a.value + b.value)
        self.a, self.b = a, b
        self.dependencies = {self}.union(a.dependencies).union(b.dependencies)

    def backpropagate_a(self):
        self.a.grad += self.grad

    def backpropagate_b(self):
        self.b.grad += self.grad


class Neg(Tensor):
    def __init__(self, a):
        super().__init__(-a.value)
        self.a = a
        self.dependencies = {self}.union(a.dependencies)
    
    def backpropagate_a(self):
        self.a.grad += -self.grad

class Mul(Tensor):
    def __init__(self, a, b):
        super().__init__(a.value * b.value)
        self.a, self.b = a, b
        self.dependencies = {self}.union(a.dependencies).union(b.dependencies)

    def backpropagate_a(self):
        gradient = self.grad * self.b.value
        if gradient.shape == self.a.grad.shape:
            # print('adding to a...', type(self.a), gradient)
            self.a.grad += gradient
        else:
            self.a.grad += gradient.sum(axis=0) 

    def backpropagate_b(self):
        gradient = self.grad * self.a.value
        if gradient.shape == self.b.grad.shape:
            # print('adding to b...', gradient)
            self.b.grad += gradient
        else:
            self.b.grad += gradient.sum(axis=0) 

class Dot(Tensor):
    def __init__(self, a, b):
        super().__init__(a.value @ b.value)
        self.a, self.b = a, b
        self.dependencies = {self}.union(a.dependencies).union(b.dependencies)

    def backpropagate_a(self):
        self.a.grad += self.grad @ self.b.value.T

    def backpropagate_b(self):
        self.b.grad += self.a.value.T @ self.grad


class Pow(Tensor):
    def __init__(self, a, b):
        super().__init__(a.value ** b.value)
        self.a, self.b = a, b
        self.dependencies = {self}.union(a.dependencies).union(b.dependencies)

    def backpropagate_a(self):
        self.a.grad += self.b.value * self.a.value ** (self.b.value-1) * self.grad

    def backpropagate_b(self):
        self.b.grad += np.log(self.a.value) * self.a.value ** self.b.value * self.grad


class Sum(Tensor):
    def __init__(self, a):
        super().__init__(sum(a.value))
        self.a = a
        self.dependencies = {self}.union(a.dependencies)

    def backpropagate_a(self):
        self.a.grad += self.grad * np.ones(self.a.value.shape)

In [44]:
np.random.seed(1)

x = Tensor(np.random.randn(5,1), requires_grad=True)
y = Tensor(np.random.randn(2,1), requires_grad=False)
# u = Tensor(np.random.randn(5,1), requires_grad=True)

m1 = Tensor(np.random.randn(4,5))
m2 = Tensor(np.random.rand(6,4))
m3 = Tensor(np.random.rand(2,6))



def eval():
    out = m1 @ x

    out = out * (out > Tensor(0))
    out = m2 @ out
    out = out * (out > Tensor(0))
    out = m3 @ out
    # # out = Tensor(np.e, requires_grad=False) ** x
    # out1 = x

    # # z = out1 - Tensor(out.value.max(), requires_grad=False)
    # out2 = Tensor(np.e, requires_grad=False) ** out1

    # out3 = out2 / Sum(out2)

    # out3 = out3 - y
    # out3 = out3 ** Tensor(2, requires_grad=False)

    # out4 = Sum(out3)

    # return out1, out2, out3, out4

    # out = Tensor(np.e, requires_grad=False) ** out
    
    out = out * Tensor(1, requires_grad=False)

    out = (out) * Sum(out)

    

    out = (out - y) ** Tensor(2, requires_grad=False)

    out = Sum(out)

    return out

# x.grad = np.zeros(x.value.shape)

out = eval()

before = out.value
out.backwards()

delta = 0.0000000001
nudge = np.zeros(m1.value.shape)
nudge[1,1] = delta

m1.value = m1.value + nudge

out = eval()
after = out.value

print('Real gradient:', (after - before) / delta)

# out.backwards()
m1.grad[1,1]


Real gradient: [-292.87406278]


-292.8746565746313

In [20]:
out.a.b.dependencies

{<__main__.Sum at 0x20f2fc795d0>,
 <__main__.Tensor at 0x20f180fef50>}

In [35]:
out.add_to_queue(out.a, [])

In [3]:
out.a.a.b.value * out.a.a.grad

array([[0.55371241],
       [0.55371241],
       [0.55371241],
       [0.55371241],
       [0.55371241]])

In [72]:
(out.a.a.b.value * out.a.a.grad).shape == out.a.a.a.grad.shape

True

In [78]:
out.a.a.a.grad

array([[-0.60799791],
       [-0.60799791],
       [-0.60799791],
       [-0.60799791],
       [-0.60799791]])

In [23]:
queue = [1,2,3,4]
el = 1.5

queue[:1] + [el] + queue[1:]

[1, 1.5, 2, 3, 4]

In [69]:
x = Tensor([1,2,3], requires_grad=True)


def eval():
    # out = m1 @ x

    # out = out * (out > Tensor(0))
    # out = m2 @ out
    # out = out * (out > Tensor(0))
    # out = m3 @ out
    out = x
    z = out - Tensor(out.value.max(), requires_grad=False)
    exps = Tensor(np.e, requires_grad=False) ** z
    # out = x * y
    out = exps * Sum(exps)
    # out = out - y
    # out = out ** Tensor(2, requires_grad=False)

    return out


out = eval()

before = out.value
out.backwards()

delta = 0.0000000001
nudge = np.zeros(x.value.shape)
nudge[1] = delta

x.value = x.value + nudge

out = eval()
after = out.value

print((after - before) / delta)


x.grad[1]

ValueError: non-broadcastable output operand with shape () doesn't match the broadcast shape (3,)

In [35]:
out.a.b.grad

array([1., 2., 3.])

array([[-1.61366234],
       [-0.84146798],
       [-1.24251437],
       [ 0.27741298],
       [ 0.44278722]])

In [54]:
x = np.array([1,2,3])
y = np.array([1,2,3])

out = sum(sum(y) * x)
before = out

nudge = np.zeros(x.shape)
delta = 0.00001
nudge[0] = delta
y = y + nudge

out = sum(sum(y) * x)
after = out

(after - before) / delta

5.999999999772853

In [47]:
.0001200001 / delta


1200001.0

In [43]:
before

49